In [ ]:
# =====================================
# 🛠 Enhanced Modular Data Collection
# =====================================

import logging
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os

# Import our enhanced utilities
from app_config import Config
from enhanced_breeze_utils import EnhancedBreezeDataManager, OptionChainAnalyzer
from data_processing_utils import TechnicalIndicatorProcessor, OptionsDataProcessor

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

logger = logging.getLogger(__name__)

# =====================================
# 🛠 Initialize Enhanced Data Manager
# =====================================

try:
    # Initialize configuration and enhanced data manager
    config = Config()
    data_manager = EnhancedBreezeDataManager()
    
    # Initialize processing utilities
    indicator_processor = TechnicalIndicatorProcessor()
    options_processor = OptionsDataProcessor()
    option_analyzer = OptionChainAnalyzer()
    
    # Set up Google Drive if in Colab environment
    try:
        from google.colab import drive
        drive.mount('/content/drive')
        logger.info("✅ Google Drive mounted")
    except ImportError:
        logger.info("ℹ️ Not in Colab environment, skipping Google Drive mount")
    
    # Authenticate with enhanced retry logic
    auth_result = data_manager.authenticate()
    if auth_result.success:
        logger.info("✅ Breeze API authenticated successfully")
        breeze = data_manager.breeze
    else:
        logger.error(f"❌ Authentication failed: {auth_result.error_message}")
        raise Exception(f"Authentication failed: {auth_result.error_message}")
        
except Exception as e:
    logger.error(f"Critical initialization error: {str(e)}")
    raise

logger.info("✅ All modules and enhanced utilities loaded successfully!")

In [ ]:
# =====================================
# 🛠 Parameter Setup with Validation
# =====================================

from enhanced_breeze_utils import MarketDataRequest
from data_processing_utils import ValidationError

def setup_trading_parameters():
    """Setup and validate trading parameters with proper error handling"""
    try:
        # Basic parameters
        stock_name = "TCS"
        interval = "5minute"
        
        # Get trading dates using enhanced utilities
        date_result = data_manager.get_trading_dates(days_back=30)
        if not date_result.success:
            raise ValidationError(f"Failed to get trading dates: {date_result.error_message}")
        
        from_date = date_result.data['from_date']
        to_date = date_result.data['to_date']
        
        logger.info(f"📅 Trading period: {from_date} to {to_date}")
        
        # Get current LTP with enhanced error handling
        ltp_result = data_manager.get_live_price(stock_name, "NSE")
        if not ltp_result.success:
            raise ValidationError(f"Failed to get LTP: {ltp_result.error_message}")
        
        ltp = ltp_result.data['ltp']
        logger.info(f"📦 Current LTP for {stock_name}: {ltp}")
        
        # Get valid expiry using enhanced option analyzer
        expiry_result = option_analyzer.get_next_valid_expiry(stock_name)
        if not expiry_result.success:
            raise ValidationError(f"Failed to get expiry: {expiry_result.error_message}")
        
        expiry_date = expiry_result.data['expiry_date']
        logger.info(f"📌 Using expiry: {expiry_date}")
        
        # Create structured request object
        request = MarketDataRequest(
            stock_code=stock_name,
            exchange_code="NSE",
            interval=interval,
            from_date=from_date,
            to_date=to_date,
            expiry_date=expiry_date,
            current_price=ltp
        )
        
        return request
        
    except Exception as e:
        logger.error(f"Parameter setup failed: {str(e)}")
        raise

# Setup parameters
market_request = setup_trading_parameters()
logger.info("✅ Parameters setup completed successfully")

In [ ]:
# =====================================
# 📈 Fetch Equity Data
# =====================================

def fetch_equity_data(request):
    """Fetch equity data with comprehensive error handling"""
    try:
        logger.info(f"📊 Fetching equity data for {request.stock_code}")
        
        # Use enhanced data manager for equity data
        equity_result = data_manager.fetch_historical_data(
            stock_code=request.stock_code,
            exchange_code=request.exchange_code,
            product_type="cash",
            interval=request.interval,
            from_date=request.from_date,
            to_date=request.to_date
        )
        
        if not equity_result.success:
            raise ValidationError(f"Equity data fetch failed: {equity_result.error_message}")
        
        equity_df = equity_result.data
        
        # Process with technical indicators using enhanced processor
        processing_result = indicator_processor.process_dataframe(
            equity_df,
            add_all_indicators=True
        )
        
        if not processing_result.success:
            logger.warning(f"Technical indicator processing had issues: {processing_result.error_message}")
            # Continue with raw data if indicator processing fails
            processed_df = equity_df
        else:
            processed_df = processing_result.data
        
        # Save with metadata
        save_result = data_manager.save_dataframe(
            processed_df,
            "tcs_equity_data.csv",
            metadata={
                "source": "equity",
                "stock_code": request.stock_code,
                "interval": request.interval,
                "indicators_count": len([c for c in processed_df.columns if c not in ['datetime', 'open', 'high', 'low', 'close', 'volume']])
            }
        )
        
        if save_result.success:
            logger.info(f"✅ Equity data saved: {len(processed_df)} records with {len(processed_df.columns)} features")
        else:
            logger.warning(f"Save failed: {save_result.error_message}")
        
        return processed_df
        
    except Exception as e:
        logger.error(f"Equity data fetch failed: {str(e)}")
        raise

# Fetch equity data
equity_df = fetch_equity_data(market_request)
logger.info(f"📈 Equity data shape: {equity_df.shape}")

In [ ]:
# =====================================
# 📊 Fetch Futures Data
# =====================================

def fetch_futures_data(request):
    """Fetch futures data with comprehensive error handling"""
    try:
        logger.info(f"📊 Fetching futures data for {request.stock_code}")
        
        # Use enhanced data manager for futures data
        futures_result = data_manager.fetch_historical_data(
            stock_code=request.stock_code,
            exchange_code="NFO",
            product_type="futures",
            interval=request.interval,
            from_date=request.from_date,
            to_date=request.to_date,
            expiry_date=request.expiry_date
        )
        
        if not futures_result.success:
            logger.warning(f"Futures data fetch failed: {futures_result.error_message}")
            return None  # Return None instead of failing completely
        
        futures_df = futures_result.data
        
        # Process with technical indicators
        processing_result = indicator_processor.process_dataframe(
            futures_df,
            add_all_indicators=True
        )
        
        if not processing_result.success:
            logger.warning(f"Futures technical indicator processing failed: {processing_result.error_message}")
            processed_df = futures_df
        else:
            processed_df = processing_result.data
        
        # Save with metadata
        save_result = data_manager.save_dataframe(
            processed_df,
            "tcs_futures_data.csv",
            metadata={
                "source": "futures",
                "stock_code": request.stock_code,
                "expiry_date": request.expiry_date,
                "interval": request.interval
            }
        )
        
        if save_result.success:
            logger.info(f"✅ Futures data saved: {len(processed_df)} records")
        else:
            logger.warning(f"Futures save failed: {save_result.error_message}")
        
        return processed_df
        
    except Exception as e:
        logger.error(f"Futures data processing error: {str(e)}")
        return None  # Graceful degradation

# Fetch futures data
futures_df = fetch_futures_data(market_request)
if futures_df is not None:
    logger.info(f"📊 Futures data shape: {futures_df.shape}")
else:
    logger.warning("⚠️ Futures data not available, continuing without it")

In [ ]:
# =====================================
# 🔄 Fetch Options Data
# =====================================

def fetch_options_data(request):
    """Fetch comprehensive options data with enhanced error handling"""
    try:
        logger.info(f"🔄 Fetching options chain for {request.stock_code}")
        
        # Use enhanced option analyzer for comprehensive chain data
        chain_result = option_analyzer.fetch_full_option_chain(
            stock_code=request.stock_code,
            expiry_date=request.expiry_date,
            current_price=request.current_price,
            interval=request.interval,
            from_date=request.from_date,
            to_date=request.to_date,
            strike_range=800  # Configurable range
        )
        
        if not chain_result.success:
            logger.warning(f"Options chain fetch failed: {chain_result.error_message}")
            return None
        
        options_df = chain_result.data
        
        # Process options with specialized processor
        processing_result = options_processor.process_options_dataframe(
            options_df,
            current_price=request.current_price,
            add_greeks=True,
            add_technical_indicators=True
        )
        
        if not processing_result.success:
            logger.warning(f"Options processing failed: {processing_result.error_message}")
            processed_df = options_df
        else:
            processed_df = processing_result.data
        
        # Save with comprehensive metadata
        save_result = data_manager.save_dataframe(
            processed_df,
            "tcs_options_data.csv",
            metadata={
                "source": "options",
                "stock_code": request.stock_code,
                "expiry_date": request.expiry_date,
                "current_price": request.current_price,
                "strike_count": len(processed_df['strike'].unique()) if 'strike' in processed_df.columns else 0,
                "total_records": len(processed_df)
            }
        )
        
        if save_result.success:
            unique_strikes = len(processed_df['strike'].unique()) if 'strike' in processed_df.columns else 0
            logger.info(f"✅ Options data saved: {len(processed_df)} records, {unique_strikes} strikes")
        else:
            logger.warning(f"Options save failed: {save_result.error_message}")
        
        return processed_df
        
    except Exception as e:
        logger.error(f"Options data processing error: {str(e)}")
        return None

# Fetch options data
options_df = fetch_options_data(market_request)
if options_df is not None:
    logger.info(f"🔄 Options data shape: {options_df.shape}")
else:
    logger.warning("⚠️ Options data not available, continuing without it")

In [ ]:
# =====================================
# 🔗 Data Combination and Enhancement
# =====================================

from data_processing_utils import ProcessingResult, DataQuality

def combine_and_enhance_data(equity_df, futures_df=None, options_df=None):
    """Combine and enhance all datasets with comprehensive error handling"""
    try:
        logger.info("🔗 Starting data combination and enhancement")
        
        # Use enhanced options processor for data combination
        combination_result = options_processor.combine_market_data(
            equity_data=equity_df,
            futures_data=futures_df,
            options_data=options_df
        )
        
        if not combination_result.success:
            raise ValidationError(f"Data combination failed: {combination_result.error_message}")
        
        combined_df = combination_result.data
        logger.info(f"✅ Data combined successfully: {combined_df.shape}")
        
        # Enhance with relationship metadata using options processor
        enhancement_result = options_processor.add_relationship_features(
            combined_df,
            include_correlations=True,
            include_price_targets=True
        )
        
        if not enhancement_result.success:
            logger.warning(f"Enhancement failed: {enhancement_result.error_message}")
            enhanced_df = combined_df
        else:
            enhanced_df = enhancement_result.data
        
        # Data quality assessment
        quality_result = options_processor.assess_data_quality(enhanced_df)
        logger.info(f"📊 Data quality: {quality_result.metadata.get('quality_score', 'N/A')}")
        
        # Save final enhanced dataset
        save_result = data_manager.save_dataframe(
            enhanced_df,
            "tcs_enhanced_data.csv",
            metadata={
                "source": "combined_enhanced",
                "features_count": len(enhanced_df.columns),
                "records_count": len(enhanced_df),
                "data_quality": quality_result.metadata.get('quality_score'),
                "processing_timestamp": datetime.now().isoformat()
            }
        )
        
        if save_result.success:
            logger.info(f"✅ Enhanced dataset saved: {enhanced_df.shape} with {len(enhanced_df.columns)} features")
        else:
            logger.warning(f"Enhanced data save failed: {save_result.error_message}")
        
        return enhanced_df
        
    except Exception as e:
        logger.error(f"Data combination and enhancement failed: {str(e)}")
        raise

# Combine and enhance all data
try:
    enhanced_df = combine_and_enhance_data(equity_df, futures_df, options_df)
    
    # Final data summary
    logger.info("="*50)
    logger.info("📊 FINAL DATA SUMMARY")
    logger.info("="*50)
    logger.info(f"📈 Total records: {len(enhanced_df):,}")
    logger.info(f"📊 Total features: {len(enhanced_df.columns):,}")
    logger.info(f"📅 Date range: {enhanced_df['datetime'].min()} to {enhanced_df['datetime'].max()}")
    
    # Feature breakdown
    equity_features = len([c for c in enhanced_df.columns if c.startswith('equity_')])
    futures_features = len([c for c in enhanced_df.columns if c.startswith('futures_')])
    options_features = len([c for c in enhanced_df.columns if c.startswith('options_')])
    relationship_features = len([c for c in enhanced_df.columns if any(keyword in c for keyword in ['corr_', 'basis_', 'divergence', 'ratio'])])
    
    logger.info(f"📈 Equity features: {equity_features}")
    logger.info(f"📊 Futures features: {futures_features}")
    logger.info(f"🔄 Options features: {options_features}")
    logger.info(f"🔗 Relationship features: {relationship_features}")
    logger.info("="*50)
    logger.info("✅✅✅ ALL DATA PROCESSING COMPLETED SUCCESSFULLY!")
    logger.info("="*50)
    
except Exception as e:
    logger.error(f"❌ Critical error in data processing: {str(e)}")
    # Provide graceful fallback
    if 'equity_df' in locals():
        logger.info("📈 Falling back to equity data only")
        enhanced_df = equity_df
    else:
        logger.error("💥 Complete failure - no data available")
        raise